# GenAI with Python: Coding Agents

###### [Article: TowardsDataScience]()

### DB

In [69]:
import pandas as pd

dtf = pd.read_csv("data_titanic.csv")
dtf.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [70]:
print("How many people died? -->", len(dtf[dtf["Survived"]==0]))

How many people died? --> 549


In [71]:
import sqlite3

dtf.to_sql(index=False, name="titanic", con=sqlite3.connect("database.db"), if_exists="replace")

891

In [72]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///database.db")

### LLM

In [73]:
from langchain_community.llms import Ollama #0.2.12

llm = Ollama(model="llama3.1")
res = llm.invoke(input=["Do you know how to code?"]).split("\n")[0]
print(res)

I have been trained on a wide range of programming languages and can understand code in many different formats. However, I don't have personal experience or the ability to write code from scratch like a human developer would.


### 1 - SQL Agent

##### Tools

In [74]:
## DB Connection
from crewai_tools import tool #0.8.3
from langchain_community.tools.sql_database.tool import ListSQLDatabaseTool, InfoSQLDatabaseTool, QuerySQLDataBaseTool

@tool("tool_tables")
def tool_tables() -> str:
    """Get all the tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool("tool_schema")
def tool_schema(tables: str) -> str:
    """
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

@tool("tool_query")
def tool_query(sql: str) -> str:
    """Execute a SQL query on the database and return the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql)

print("--- Get Tables ---")
print( tool_tables.run() )

print("\n--- Get Schema ---")
print( tool_schema.run( tables=tool_tables.run() ) )

print("\n--- SQL Query ---")
print( tool_query.run(f"SELECT * FROM {tool_tables.run()} LIMIT 3") )

--- Get Tables ---
Using Tool: tool_tables
titanic

--- Get Schema ---
Using Tool: tool_tables
Using Tool: tool_schema

CREATE TABLE titanic (
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/

--- SQL Query ---
Using Tool: tool_tables
Using Tool: tool_query
[(1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0, 'A/5 21171', 7.25, None, 'S'), (2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C'), (3, 1, 3, 'Hei

In [75]:
## LLM Checking SQL
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool

@tool("tool_check")
def tool_check(sql: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. 
    Always use this tool before executing a query with `tool_query`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query":sql})

tool_check.run(f"SELECT * FROM {tool_tables.run()} LIMIT 3 +db+")

Using Tool: tool_tables
Using Tool: tool_check


'SELECT * FROM titanic LIMIT 3'

##### Agent

In [76]:
import crewai #0.51.1

prompt = '''Construct and execute SQL queries based on {user_input}'''

## Agent
agent_sql = crewai.Agent(
    role="Database Engineer",
    goal=prompt,
    backstory='''
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `tool_tables` to find available tables.
        Use the `tool_schema` to understand the metadata for the tables.
        Use the `tool_query` to check your queries for correctness.
        Use the `tool_check` to execute queries against the database.
     ''',
    tools=[tool_tables, tool_schema, tool_query, tool_check], 
    max_iter=3,
    llm=llm,
    allow_delegation=False, verbose=False)

## Task
task_sql = crewai.Task(
    description=prompt,
    agent=agent_sql,
    expected_output='''Database result from the query''')

##### Test single Agent

In [77]:
crew = crewai.Crew(agents=[agent_sql], tasks=[task_sql], verbose=False)
res = crew.kickoff(inputs={"user_input":"how many people died?"})
print("\n---Res---\n", res)

2024-08-27 00:43:04,800 - 140704473259456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed



---Res---
 Agent stopped due to iteration limit or time limit.


### 2 - Python Agent

##### Tools

##### Agent

In [78]:
prompt = '''You receive data from the database engineer and analyze it to answer {user_input}'''

## Agent
agent_py = crewai.Agent(
    role="Data Analyst",
    goal=prompt,
    backstory='''
        You are an experienced data anlyst that analyzes datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. 
        You have attention to detail and always produce very detailed work.
     ''',
    #tools=[], 
    max_iter=3,
    llm=llm,
    allow_delegation=False, verbose=False)

## Task
task_py = crewai.Task(
    description=prompt,
    agent=agent_py,
    expected_output='''Detailed analysis''')

##### Test single Agent

In [79]:
crew = crewai.Crew(agents=[agent_py], tasks=[task_py], verbose=False)
res = crew.kickoff(inputs={"user_input":
'''
how many people died in this dataset?
/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
'''})
print("\n---Res---\n", res)

2024-08-27 00:46:25,886 - 140704473259456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed



---Res---
 Number of people who died: 1


### 3 - HTML Agent

##### Tools

##### Agent

In [80]:
prompt = '''You write an executive summary type of report based on the work of the analyst to answer {user_input}'''

## Agent
agent_html = crewai.Agent(
    role="Reporting Specialist",
    goal=prompt,
    backstory='''
        You are an experienced reporting specialist that writes beautiful reports using HTML.
        Your writing style is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most important details.
     ''',
    #tools=[], 
    max_iter=3,
    llm=llm,
    allow_delegation=False, verbose=False)

## Task
task_html = crewai.Task(
    description=prompt,
    agent=agent_py,
    expected_output='''HTML report''')

##### Test single Agent

In [81]:
crew = crewai.Crew(agents=[agent_html], tasks=[task_html], verbose=False)
res = crew.kickoff(inputs={"user_input":
'''
how many people died in this dataset?
There was **1** person who died in this dataset.

PassengerId: 1
Survived: 0
Pclass: 3
Name: Braund, Mr. Owen Harris
Sex: male
Age: 22.0
SibSp: 1
Parch: 0
Ticket: A/5 21171
Fare: 7.25
Cabin: None
Embarked: S

This is the detailed analysis of the dataset to answer how many people died in this dataset.
'''})
print("\n---Res---\n", res)

2024-08-27 00:49:55,071 - 140704473259456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed



---Res---
 <!DOCTYPE html>
<html>
<head>
	<title>Analysis Report: How Many People Died in this Dataset?</title>
</head>
<body>

<h1>Executive Summary Report</h1>
<p>How many people died in this dataset?</p>

<h2>Key Findings:</h2>
<ul>
  <li>There was **1** person who survived.</li>
  <li>Unfortunately, there was only **1** person who did not survive, which means 1 person died.</li>
</ul>

<p>The individual who passed away had:
- PassengerId: 1
- Survived: 0 (indicating they did not survive the event)
- Pclass: 3 (the passenger class)
- Name: Braund, Mr. Owen Harris
- Sex: male
- Age: 22.0
- SibSp: 1 (number of siblings/spouses aboard)
- Parch: 0 (number of parents/children aboard)
- Ticket: A/5 21171
- Fare: 7.25 (the ticket fare)
- Cabin: None (no cabin was allocated to the passenger)
- Embarked: S (the location where the passenger boarded)</p>

<h2>Conclusion:</h2>
<p>In conclusion, there was only **1** person who died in this dataset.</p>

<h3>Summary Tables</h3>
<h4>Table 1: Summ

### 4 - Run all

In [82]:
crew = crewai.Crew(agents=[agent_sql, agent_py, agent_html], 
                   tasks=[task_sql, task_py, task_html], 
                   process=crewai.Process.sequential,
                   verbose=False)

res = crew.kickoff(inputs={"user_input":"how many people died?"})

print("\n---Res---\n", res)

2024-08-27 00:54:14,814 - 140704473259456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed

KeyboardInterrupt



<div style="text-align: center;">
  <h1>Total Number of Deaths</h1>
  <p>Based on the analysis, it was determined that:</p>
</div>

**<div style="background-color: #f0f0f0; border: 1px solid #ccc; padding: 10px;">**

| **Total Number of Deaths** | **Description** |
| --- | --- |
| <font size=6>100,456</font> | The total number of deaths calculated from the dataset. |

**</div>**

### 5 - Machine Learning

In [ ]:
prompt = '''You take the dataframe from the engineer and make a prediction using sckit-learn to answer {user_input}'''

## Agent
agent_ml = crewai.Agent(
    role="Data Scientist",
    goal=prompt,
    backstory='''
        You are an experienced data scientist that does machine learning using Python and sckit-learn.
        Take the dataframe and split into train set and test set and train a simple classification to predict a column.
        Then you use the score to evaluate the model predictions.
     ''',
    #tools=[], 
    #max_iter=3,
    llm=llm,
    allow_delegation=False, verbose=True)

## Task
task_ml = crewai.Task(
    description=prompt,
    agent=agent_py,
    expected_output='''Prediction of the machine learnig model''')

In [ ]:
crew = crewai.Crew(agents=[agent_sql, agent_ml], 
                   tasks=[task_sql, task_ml], 
                   process=crewai.Process.sequential,
                   verbose=False)

res = crew.kickoff(inputs={"user_input":"get the whole table and train a classification to predict the column `Survived`"})

print("\n---Res---\n", res)